In [14]:
from transformers import pipeline

In [15]:
# load pipeline for sentiment-analysis
sentiment_pipeline = pipeline('sentiment-analysis')

# uji coba
data = ["This movie is fantastic, I love it!", "The plot was boring and the acting was terrible."]
print(sentiment_pipeline(data))


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9998769760131836}, {'label': 'NEGATIVE', 'score': 0.9997890591621399}]


In [2]:
from datasets import load_dataset
imdb = load_dataset("imdb")

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=256)
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
)

trainer.train()

c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
500,0.425100
1000,0.373000
1500,0.339400
2000,0.322400
2500,0.312200
3000,0.306100
3500,0.229800
4000,0.188500
4500,0.199100
5000,0.198500


c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=9375, training_loss=0.206001811319987, metrics={'train_runtime': 39778.5009, 'train_samples_per_second': 1.885, 'train_steps_per_second': 0.236, 'total_flos': 4967527449600000.0, 'train_loss': 0.206001811319987, 'epoch': 3.0})

In [6]:
trainer.evaluate()

c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.45617571473121643,
 'eval_runtime': 3301.2143,
 'eval_samples_per_second': 7.573,
 'eval_steps_per_second': 0.947,
 'epoch': 3.0}

In [14]:
trainer.save_model("./test_trainer")
tokenizer.save_pretrained("./test_trainer")


('./test_trainer\\tokenizer_config.json',
 './test_trainer\\special_tokens_map.json',
 './test_trainer\\vocab.txt',
 './test_trainer\\added_tokens.json',
 './test_trainer\\tokenizer.json')

In [16]:
from transformers import pipeline

# Muat model yang sudah Anda fine-tune dari folder output
# (Folder "test_trainer" dibuat secara otomatis oleh Trainer)
sentiment_model = pipeline(task="sentiment-analysis", model="./test_trainer")

# Buat beberapa kalimat baru untuk diuji
kalimat_uji = [
    "The story was predictable, but the visuals were stunning.",
    "I would not recommend this movie to anyone.",
    "A true masterpiece of cinema, absolutely brilliant!",
    "Filmnya biasa saja, tidak ada yang spesial." # Coba dengan Bahasa Indonesia!
]

# Lakukan prediksi
hasil = sentiment_model(kalimat_uji)
print(hasil)

Device set to use cpu


[{'label': 'LABEL_1', 'score': 0.9946309328079224}, {'label': 'LABEL_0', 'score': 0.9996973276138306}, {'label': 'LABEL_1', 'score': 0.9993736147880554}, {'label': 'LABEL_0', 'score': 0.976233184337616}]


In [18]:
trainer.save_model("./model-sentimen-imdb-final")